# Отслеживание объектов на ленте конвейера
**Хакатон компании Ренью**

**Задача**:  Создать трекер для отслеживания движущихся объектов на ленте конвейера мусороперерабатывающего завода. (пластиковые бутылки разных типов)
  
**Заказчик**: Renue, IT-компания г. Екатеринбург

**Сроки проекта**: 19/08/24 - 09/09/24.

**Стек технологий**: cv2, ultralytics, YOLO8, BoT-SORT, ByteTrack, SORT, DeepSORT.

### Описание:
На мусороперерабатывающем заводе над конвейерной лентой установлена камера, которая фиксирует движение пластикового мусора. Данные в потоке передаются детектору и трекеру, которые определяют тип мусора и координаты bounding box.
  
Необходимо улучшить работу трекера:
  - получение более точных координат bounding box;
  - обеспечить устойчивость прослеживания объекта без смены ID;

### Требования заказчика:
* в течении 2х недель разработать решение для отслеживания объектов на ленте конвейера
* скорость обработки должна быть не более 100мс на кадр
* добиться наилучшего значения метрики MOTA
* подготовить отчет о работе.

### Исходные данные:
* Примеры видеозаписей работы конвейера
* Модель детекции и код для ее запуска (пример)
* Датасет в нескольких форматах: MOT, COCO, CVAT
  - изображения
  - разметка
* Ссылка на данные https://...

### Состав команды:
Альбина,  @AlbinaUsaeva  
Татьяна, @Tanya_GileT  
Павел, @keyboardnorth  




# Установка библиотек

# Загрузка библиотек

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from tqdm.notebook import tqdm

from time import time
import datetime
from datetime import timedelta

from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
from deep_sort_realtime.deepsort_tracker import Tracker
import motmetrics as mm

import sys
import os

In [ ]:
sys.path.append(os.path.dirname('/kaggle/input/trackers/sort_for_Kaggle.py'))
from sort_for_Kaggle import Sort

# Константы

In [ ]:
# задание константных значений
CONFIDENCE_THRESHOLD = 0.7
IOU = 0.3
MAX_AGE = 60

In [ ]:
# переменная для обращения к видео
#video_path = "/content/31-03-2024-09%3A34%3A24.mp4"
video_path = "/kaggle/input/tracking-of-bottles/Videos/Videos/31-03-2024-09%3A34%3A24.mp4"

# Создание необходимых функций

In [ ]:
# функция для записи видео
def create_video_writer(cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [ ]:
# функция для расчета метрики MOTA

def motMetricsEnhancedCalculator(gtSource, tSource, min_fr=1, max_fr=101):
    '''вычисляет метрику MOTA; на входе путь к размеченному датасету и с новой разметкой'''
    # load ground truth
    gt = np.loadtxt(gtSource, delimiter=',')

    # load tracking output
    try:
        t = np.loadtxt(tSource, delimiter=',')
    except:
        t = tSource

    # Create an accumulator that will be updated during each frame
    acc = mm.MOTAccumulator(auto_id=True)

    # Max frame number maybe different for gt and t files
    #for frame in range(1, int(gt[:,0].max())+1):      # detection and frame numbers begin at 1
    for frame in range(min_fr,max_fr):    # задаём конкретные фреймы

        # select id, x, y, width, height for current frame
        # required format for distance calculation is X, Y, Width, Height \
        # We already have this format
        gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
        t_dets  =  t[t [:,0]==frame,1:6] # select all detections in t

        C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                    max_iou=0.5) # format: gt, t

        # Call update once for per frame.
        # format: gt object ids, t object ids, distance
        acc.update(gt_dets[:,0].astype('int').tolist(), \
                  t_dets[:,0].astype('int').tolist(), C)

    mh = mm.metrics.create()

    return mh.compute(acc, metrics=['num_frames', 'recall', 'precision', 'num_objects',
                                    'num_switches', 'mota', 'motp'], name='acc')

# Разработка базового решения
- вывод на экран результатов работы встроенного трекера YOLO на 10 кадрах
- тестирование работы различных трекеров на 100 фреймах, выбор трекера с наилучшим результатов
- подбор гиперпараметров наилучшего трекера
- расчет финальной метрики на 9000 фреймах, запись видео с результатом.

# Тестирование трекеров: botsort, bytetrack, deepsort
Сначала посмотрим как работает базовое решение на 10 кадрах.

In [ ]:
# проходимся циклом по видео, для захвата 10 фреймов и отрисовки работы детектора и трекера

cap = cv2.VideoCapture(video_path) # инициализируем объект для захвата видео
#model = YOLO("/content/yolov10x_v2_4_best.pt")   # загружаем модель
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")   # загружаем модель


track_history = defaultdict(lambda: [])
track_data = np.zeros((1,6))

for i in range(1, 10):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)

    success, frame = cap.read()
    if success:
        results = model.track(frame, iou=IOU, persist=True, conf=CONFIDENCE_THRESHOLD, verbose=False)
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu().tolist()
        annotated_frame = results[0].plot()

        #  Отрисовка ббоксов
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
               track.pop(0)

            # Отрисовка линий трекера
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            track_path = cv2.polylines(annotated_frame, [points],
                                       isClosed=False, color=(230, 230, 230),
                                       thickness=10)

            track_data_new = np.hstack([i, track_id, (x-w/2), (y-h/2), w, h])
            track_data = np.vstack([track_data, track_data_new])
        cv2_imshow(annotated_frame)
        plt.imshow(annotated_frame)
        plt.axis('off')
        plt.show()

        if cv2.waitKey(1) & 0xFF == ord("q"):
           break
    else:
        break
cap.release()
cv2.destroyAllWindows()

track_data = np.delete(track_data, 0,0)

**Тестируем работу различных трекеров на 100 фреймах**

## BaseLine - трекер botsort

In [ ]:
# проходимся циклом по видео, для захвата 100 фреймов и расчета первичной метрики для трекера botsort

cap = cv2.VideoCapture(video_path) # инициализируем объект для захвата видео
#model = YOLO("/content/yolov10x_v2_4_best.pt")   # загружаем модель
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")   # загружаем модель
tracker="botsort.yaml"    # загружаем трекер

track_history = defaultdict(lambda: [])
track_data = np.zeros((1,6))

for i in tqdm(range(1, 101)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)

    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, conf=0.7, iou=0.45, verbose=False)
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu().tolist()
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
               track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            track_path = cv2.polylines(annotated_frame, [points],
                                       isClosed=False, color=(230, 230, 230),
                                       thickness=10)

            track_data_new = np.hstack([i, track_id, (x-w/2), (y-h/2), w, h])
            track_data = np.vstack([track_data, track_data_new])
        writer_yolo.write(annotated_frame)
        writer_yolo.write(track_path)
        cv2_imshow(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
           break
    else:
        break
cap.release()
writer_yolo.release()
cv2.destroyAllWindows()

track_data = np.delete(track_data, 0,0)

In [ ]:
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt',
                              track_data, min_fr=1, max_fr=101)

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,100,0.921241,1.0,419,2,0.916468,0.088559


## BaseLine - трекер bytetrack

In [ ]:
# проходимся циклом по видео, для захвата 100 фреймов и расчета первичной метрики bytetrack

cap = cv2.VideoCapture(video_path) # инициализируем объект для захвата видео
#model = YOLO("/content/yolov10x_v2_4_best.pt")   # загружаем модель
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")   # загружаем модель
tracker="bytetrack.yaml"    # загружаем трекер

track_history = defaultdict(lambda: [])
track_data = np.zeros((1,6))

for i in tqdm(range(1, 101)):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)

    success, frame = cap.read()
    if success:
        results = model.track(frame, tracker=tracker, persist=True, conf=0.7, iou=0.45, verbose=False)
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        confidences = results[0].boxes.conf.cpu().tolist()
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
               track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            track_path = cv2.polylines(annotated_frame, [points],
                                       isClosed=False, color=(230, 230, 230),
                                       thickness=10)

            track_data_new = np.hstack([i, track_id, (x-w/2), (y-h/2), w, h])
            track_data = np.vstack([track_data, track_data_new])
        writer_yolo.write(annotated_frame)
        writer_yolo.write(track_path)
        cv2_imshow(annotated_frame)
        plt.imshow(annotated_frame)
        plt.axis('off')
        plt.show()

        if cv2.waitKey(1) & 0xFF == ord("q"):
           break
    else:
        break
cap.release()
writer_yolo.release()
cv2.destroyAllWindows()

track_data = np.delete(track_data, 0,0)

In [ ]:
# Расчитываем метрику для bytetrack
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt', track_data)

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,100,0.701671,0.763636,419,2,0.479714,0.258718


## Трекер DeepSORT

**Произведем отрисовку ббоксов на 10 фреймах.**

In [ ]:
# инициализируем объект для захвата видео
cap = cv2.VideoCapture(video_path)  # видео основное

# инициализируем объект для записи видео
writer_deepsort = create_video_writer(cap, "output_deepsort.mp4")

# загружаем предобученную модель YOLOv8n
#model = YOLO("/content/yolov10x_v2_4_best.pt")
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
# создаем массив для записи в него данных для расчета метрики
track_data = np.zeros((1, 6))
track_history = defaultdict(lambda: [])

tracker = DeepSort(max_age=MAX_AGE, max_iou_distance=0.8)

# # инициализируем цветовую карту
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# запускаем цикл по нужному количеству кадров видео
for i in tqdm(range(1, 11)):

    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    start = datetime.datetime.now()

    ret, frame = cap.read()

    if not ret:
        break

    # запускаем модель YOLO на фрейме
    detections = model(frame, verbose=False)[0]

    # создаем список для записи результатов детекции
    results = []

    # проходим в цикле по каждому результату детекции
    for data in detections.boxes.data.tolist():
        # получаем вероятность существования данного объекта
        confidence = data[4]

        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # если вероятность выше, чем граничная получаем координаты bbox и class_id
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])

        results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    # обновляем данные трекера
    tracks = tracker.update_tracks(results, frame=frame)

    # запускаем цикл по трекам
    for track in tracks:

        # получаем track id
        track_id = track.track_id
        # получаем координаты bbox лево-верх и право-низ
        tlbr = track.to_tlbr(orig=True)

        score = track.det_conf
        class_id = track.det_class

        if score is None or score < CONFIDENCE_THRESHOLD:
             continue

        xmin, ymin, xmax, ymax = int(tlbr[0]), int(tlbr[1]), int(tlbr[2]), int(tlbr[3])

        track_data_new = np.hstack([i, track_id, xmin, ymin, (xmax-xmin), (ymax-ymin)])
        track_data = np.vstack([track_data, track_data_new])

        # рисуем bbox и добавляем подписи
        color = colors[int(track_id) % len(colors)]
        color = [i * 255 for i in color]
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 3)
        cv2.rectangle(frame, (xmin, ymin-30), (xmax, ymin), color, -1)
        cv2.putText(frame,
                    "track_id: " + str(track_id) + "-" + "class " + str(class_id),
                     (xmin, ymin-10), 0, 0.75, (255,255,255), 2)

        # находим координаты центра bbox
        cx, cy = int((xmin+xmax)/2), int((ymin+ymax)/2)

        track_info = track_history[track]
        track_info.append((cx, cy))
        if len(track_info) > 30:
            track_info.pop(0)

        # получаем линию трека
        points = np.hstack(track_info).astype(np.int32).reshape((-1, 1, 2))
        track_path = cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # завершение фиксации времени
    end = datetime.datetime.now()
    # вывод времени обработки одного фрейма
    # print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
    # расчет количества фреймов в секунду и вывод результата
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    #cv2_imshow(frame)
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

    writer_deepsort.write(frame)
    writer_deepsort.write(track_path)
    #if cv2.waitKey(1) == ord("q"):
    #    break

cap.release()
writer_deepsort.release()
cv2.destroyAllWindows()

# удаляем строчку с нулями
track_data = np.delete(track_data, 0,0)

In [ ]:
# расчет метрик для трекера DeepSORT на 10 фреймах
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt', track_data.astype('int'),
                             min_fr=1, max_fr=11)

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,10,1.0,1.0,32,1,0.96875,0.068639


**Произведем отрисовку ббоксов на 100 фреймах и расчет метрики**

In [ ]:
# инициализируем объект для захвата видео
cap = cv2.VideoCapture(video_path)  # видео основное

# инициализируем объект для записи видео
writer_deepsort = create_video_writer(cap, "output_deepsort.mp4")

# загружаем предобученную модель YOLOv8n
#model = YOLO("/content/yolov10x_v2_4_best.pt")
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
# создаем массив для записи в него данных для расчета метрики
track_data = np.zeros((1, 6))
track_history = defaultdict(lambda: [])

tracker = DeepSort(max_age=MAX_AGE, max_iou_distance=0.8)

# # инициализируем цветовую карту
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# запускаем цикл по нужному количеству кадров видео
for i in tqdm(range(1, 101)):

    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    start = datetime.datetime.now()

    ret, frame = cap.read()

    if not ret:
        break

    # запускаем модель YOLO на фрейме
    detections = model(frame, verbose=False)[0]

    # создаем список для записи результатов детекции
    results = []

    # проходим в цикле по каждому результату детекции
    for data in detections.boxes.data.tolist():
        # получаем вероятность существования данного объекта
        confidence = data[4]

        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # если вероятность выше, чем граничная получаем координаты bbox и class_id
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])

        results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    # обновляем данные трекера
    tracks = tracker.update_tracks(results, frame=frame)

    # запускаем цикл по трекам
    for track in tracks:

        # получаем track id
        track_id = track.track_id
        # получаем координаты bbox лево-верх и право-низ
        tlbr = track.to_tlbr(orig=True)

        score = track.det_conf
        class_id = track.det_class

        if score is None or score < CONFIDENCE_THRESHOLD:
             continue

        xmin, ymin, xmax, ymax = int(tlbr[0]), int(tlbr[1]), int(tlbr[2]), int(tlbr[3])

        track_data_new = np.hstack([i, track_id, xmin, ymin, (xmax-xmin), (ymax-ymin)])
        track_data = np.vstack([track_data, track_data_new])

        # рисуем bbox и добавляем подписи
        color = colors[int(track_id) % len(colors)]
        color = [i * 255 for i in color]
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 3)
        cv2.rectangle(frame, (xmin, ymin-30), (xmax, ymin), color, -1)
        cv2.putText(frame,
                    "track_id: " + str(track_id) + "-" + "class " + str(class_id),
                     (xmin, ymin-10), 0, 0.75, (255,255,255), 2)

        # находим координаты центра bbox
        cx, cy = int((xmin+xmax)/2), int((ymin+ymax)/2)

        track_info = track_history[track]
        track_info.append((cx, cy))
        if len(track_info) > 30:
            track_info.pop(0)

        # получаем линию трека
        points = np.hstack(track_info).astype(np.int32).reshape((-1, 1, 2))
        track_path = cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # завершение фиксации времени
    end = datetime.datetime.now()
    # вывод времени обработки одного фрейма
    # print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
    # расчет количества фреймов в секунду и вывод результата
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    #cv2_imshow(frame)
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

    writer_deepsort.write(frame)
    writer_deepsort.write(track_path)
    #if cv2.waitKey(1) == ord("q"):
    #    break

cap.release()
writer_deepsort.release()
cv2.destroyAllWindows()

# удаляем строчку с нулями
track_data = np.delete(track_data, 0,0)

In [ ]:
# расчет метрик для трекера DeepSORT на 100 фреймах
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt', track_data.astype(int))

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,100,0.983294,0.997579,419,6,0.966587,0.055891


## Трекер SORT

In [ ]:
class ObjectDetection:

    def __init__(self, capture_index):
        self.capture_index = capture_index
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

    def load_model(self):
        model = YOLO('/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt')
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame, verbose=False)
        return results

    def get_results(self, results, fr):
        detections = []
        for result in results[0]:           # extract detections / берутся поочереди bbox'ы
            xyxy = result.boxes.xyxy.cpu().numpy()
            xywh = result.boxes.xywh.cpu().numpy()
            conf = result.boxes.conf.cpu().numpy()
            cls  = result.boxes.cls.cpu().numpy()

            merged_detection = [fr, xywh[0][0], xywh[0][1], xywh[0][2], xywh[0][3],
                                    xyxy[0][0], xyxy[0][1], xyxy[0][2], xyxy[0][3], conf[0], cls[0]]

            detections.append(merged_detection)
        return np.array(detections)

    def draw_bounding_boxes_with_id(self, img, bboxes, ids):
        for bbox, id_ in zip(bboxes, ids):
            cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0,0,255), 2)
            cv2.putText(img, "ID: "+str(id_), (int(bbox[0]), int(bbox[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        return img

    def __call__(self, max_age=100, min_hits=8, iou_threshold=0.5, fr_min=1, fr_max=11):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()

        # SORT
        sort = Sort(max_age=max_age, min_hits=min_hits, iou_threshold=iou_threshold)

        self.detections_for_mota = []

        for i in tqdm(range(fr_min, fr_max)):          # номера фреймов для обработки из видео
            cap.set(cv2.CAP_PROP_POS_FRAMES,i-1)

            ret, frame = cap.read()   # только array (картинка)
            assert ret

            results = self.predict(frame)  # передаём array, получаем общую инфу о bbox'ах, классах и др. (без id трека)

            # array; в строке по каждому bbox: фрейм/x_центр/y_центр/ширина/высота/лево/верх/право/низ/вероятность/класс
            detections = self.get_results(results, i)

            # SORT Tracking
            if len(detections) == 0:           # если на кадре нет bbox'ов - пустой array
                detections = np.empty((0,11))

            # обновляются координаты bbox (уточнение от ф.Калмана и Венгерского алг.), 5-ое значение - присвоенный трек-id
            res = sort.update(detections[:,5:10]) # подаём x_left, y_top, x_right, y_bottom, conf

            boxes_track = res[:,:-1]            # обрезается правый столбец с треком, остаются только координаты bbox
            boxes_ids = res[:,-1].astype(int)   # номера треков

            #frame = self.draw_bounding_boxes_with_id(frame, boxes_track, boxes_ids)  # в Kaggle визуализация cv2 не работает
            #cv2.imshow('tracking of plastic bottles', frame)


            detections_for_mota_fr = np.hstack([np.array([i]*len(res)).reshape(-1,1),        # frame
                                                res[:,[4,0,1]],                              # трек id, лево, верх
                                                (res[:,2]-res[:,0]).reshape(-1,1),           # ширина
                                                (res[:,3]-res[:,1]).reshape(-1,1)])          # высота

            self.detections_for_mota.append(detections_for_mota_fr)


            #if cv2.waitKey(1) & 0xFF == ord('q'):
            #    break

        self.detections_for_mota = np.vstack(self.detections_for_mota)

        cap.release()
        #cv2.destroyAllWindows()

In [ ]:
detector = ObjectDetection(capture_index='/kaggle/input/tracking-of-bottles/Videos/Videos/31-03-2024-09%3A34%3A24.mp4')

YOLOv10x summary (fused): 503 layers, 31,612,970 parameters, 0 gradients, 169.9 GFLOPs


In [ ]:
%%time
# получаем разметку с лучшими гиперпараметрами (подобраны перебором)
detector(max_age=40, min_hits=1, iou_threshold=0.3, fr_min=1, fr_max=101)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 59.4 s, sys: 17.6 s, total: 1min 16s
Wall time: 25.2 s


In [ ]:
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt', detector.detections_for_mota)

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,100,0.906921,0.984456,419,8,0.873508,0.146046


## Выбор трекера с наилучшим результатом


In [ ]:
index = ['botsort',
         'bytetrack',
         'deepsort',
         'sort']
data = {'MOTA_100f':[ 0.916468,
                      0.479714,
                      0.966587,
                      0.873508]}


scores_data = pd.DataFrame(data=data, index=index)
scores_data

,MOTA_100f
botsort,0.916468
bytetrack,0.479714
deepsort,0.966587
sort,0.873508


**Наилучший результат показала модель deepsort: MOTA = 0.966587**

# Тестирование наилучшей модели на 9000 фреймах.

In [ ]:
# инициализируем объект для захвата видео
cap = cv2.VideoCapture(video_path)  # видео основное

# инициализируем объект для записи видео
writer_deepsort = create_video_writer(cap, "output_deepsort.mp4")

# загружаем предобученную модель YOLOv8n
#model = YOLO("/content/yolov10x_v2_4_best.pt")
model = YOLO("/kaggle/input/tracking-of-bottles/Models/Models/ultralytics/yolov10x_v2_4_best.pt")

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
# создаем массив для записи в него данных для расчета метрики
track_data = np.zeros((1, 6))
track_history = defaultdict(lambda: [])

frame_time = []

tracker = DeepSort(max_age=MAX_AGE, max_iou_distance=0.8)

# # инициализируем цветовую карту
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# запускаем цикл по нужному количеству кадров видео
for i in tqdm(range(1, 9001)):

    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    start = datetime.datetime.now()

    ret, frame = cap.read()

    if not ret:
        break

    # запускаем модель YOLO на фрейме
    detections = model(frame,verbose=False)[0]

    # создаем список для записи результатов детекции
    results = []

    # проходим в цикле по каждому результату детекции
    for data in detections.boxes.data.tolist():
        # получаем вероятность существования данного объекта
        confidence = data[4]

        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # если вероятность выше, чем граничная получаем координаты bbox и class_id
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])

        results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    # обновляем данные трекера
    tracks = tracker.update_tracks(results, frame=frame)

    # запускаем цикл по трекам
    for track in tracks:

        # получаем track id
        track_id = track.track_id
        # получаем координаты bbox лево-верх и право-низ
        tlbr = track.to_tlbr(orig=True)

        score = track.det_conf
        class_id = track.det_class

        if score is None or score < CONFIDENCE_THRESHOLD:
             continue

        xmin, ymin, xmax, ymax = int(tlbr[0]), int(tlbr[1]), int(tlbr[2]), int(tlbr[3])

        track_data_new = np.hstack([i, track_id, xmin, ymin, (xmax-xmin), (ymax-ymin)])
        track_data = np.vstack([track_data, track_data_new])

        # рисуем bbox и добавляем подписи
        color = colors[int(track_id) % len(colors)]
        color = [i * 255 for i in color]
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 3)
        cv2.rectangle(frame, (xmin, ymin-30), (xmax, ymin), color, -1)
        cv2.putText(frame,
                    "track_id: " + str(track_id) + "-" + "class " + str(class_id),
                     (xmin, ymin-10), 0, 0.75, (255,255,255), 2)

        # находим координаты центра bbox
        cx, cy = int((xmin+xmax)/2), int((ymin+ymax)/2)

        track_info = track_history[track]
        track_info.append((cx, cy))
        if len(track_info) > 30:
            track_info.pop(0)

        # получаем линию трека
        points = np.hstack(track_info).astype(np.int32).reshape((-1, 1, 2))
        track_path = cv2.polylines(frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

    # завершение фиксации времени
    end = datetime.datetime.now()
    # вывод времени обработки одного фрейма
    # print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
    frame_time.append((end - start).total_seconds() * 1000)

    # расчет количества фреймов в секунду и вывод результата
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    #cv2_imshow(frame)
    #plt.imshow(frame)
    #plt.axis('off')
    #plt.show()

    writer_deepsort.write(frame)
    writer_deepsort.write(track_path)
    #if cv2.waitKey(1) == ord("q"):
    #    break

cap.release()
writer_deepsort.release()
cv2.destroyAllWindows()

# удаляем строчку с нулями
track_data = np.delete(track_data, 0,0)

  0%|          | 0/9000 [00:00<?, ?it/s]

In [ ]:
# расчет метрик для трекера DeepSORT на 9000 фреймах
motMetricsEnhancedCalculator('/kaggle/input/tracking-of-bottles/mot_dataset/gt/gt.txt', track_data.astype(int),
                             min_fr=1, max_fr=9001)

,num_frames,recall,precision,num_objects,num_switches,mota,motp
acc,9000,0.979046,0.998575,46531,1062,0.954826,0.045139


In [ ]:
pd.Series(frame_time).describe()

count    9000.000000
mean      170.403839
std        73.899234
min        72.292000
25%       119.704250
50%       147.051500
75%       205.283000
max      1769.248000
dtype: float64

## Выводы:

 Для улучшения работы трекера было протестировано 4 трекера на 100 фреймах:

| Трекер | MOTA |
| --- | --- |
| BotSORT | 0.916468 |
| ByteTrack | 0.479714 |
| SORT | 0.873508 |
| DeepSORT | 0.966587 |

 В процессе разработки решения были периодически выявленные следующие проблемы:
  - потеря объекта
  - фантомные треки
  - не корректные границы ббокса

Наилучшие показатели ключевой метрики проекта МОТА были получены трекером DeepSORT.\
**Результат MOTA на всём размеченном участке (9000 фреймов) = 0.954826.**

  